In [1]:
from paraview import simple

cone = simple.Cone()
view = simple.GetRenderView()
representation = simple.Show(cone, view)

In [2]:
from trame.app import get_server
from trame.ui.vuetify import SinglePageLayout
from trame.widgets import vuetify, paraview as pv_widgets

server = get_server(client_type="vue2")
state, ctrl = server.state, server.controller

@state.change("resolution")
def update_cone(resolution, **_):
    cone.Resolution = resolution
    ctrl.view_update()

with SinglePageLayout(server) as layout:
    with layout.toolbar:
        vuetify.VSpacer()
        vuetify.VSlider(v_model=("resolution", 12), min=3, max=60, dense=True, hide_details=True)
        with vuetify.VBtn(icon=True, click=ctrl.view_reset_camera):
            vuetify.VIcon("mdi-crop-free")
    with layout.content:
        with vuetify.VContainer(fluid=True, classes="fill-height pa-0 ma-0"):
            with pv_widgets.VtkRemoteView(view, interactive_ratio=1) as html_view:
                ctrl.view_update = html_view.update
                ctrl.view_reset_camera = html_view.reset_camera


In [3]:
await layout.ready
layout

In [4]:
# Make Output View full screen with some JS
from IPython.display import display_javascript

display_javascript("""
(function updateTrameOutputs() {
	const full = document.querySelectorAll(".jp-LinkedOutputView iframe[id^=trame]");
	const all = document.querySelectorAll("iframe[id^=trame]");
	const visibleInFullScreen = {};
	const moveToFullScreen = [];

	for (let i = 0; i < full.length; i++) {
		const iframe = full[i];
		visibleInFullScreen[iframe.id] = true;
		moveToFullScreen.push(iframe);
	}
	for (let i = 0; i < all.length; i++) {
		const iframe = all[i];
		if (moveToFullScreen.includes(iframe)) {
			continue;
		}
		if (visibleInFullScreen[iframe.id]) {
			iframe.style.display = "none";
		} else {
			iframe.style.display = "block";
		}
	}
	for (let i = 0; i < moveToFullScreen.length; i++) {
        const iframe = moveToFullScreen[i];
        iframe.style.width = "100%";
        iframe.style.height = "100%";
        iframe.parentElement.style.padding = 0;
		iframe.parentElement.parentElement.style.height = "100%";
	}
})();
""", raw=True)

In [5]:
import asyncio

async def animation():
    for i in range(3, 60):
        with state:
            state.resolution = i
        await asyncio.sleep(0.1)

await animation()